# EnvData

## _Problem:_ 
Laboratory environmental data is stored to individual loggers in a proprietary binary format. Data must be manually downloaded weekly, imported into manufaturer software, then finally exported to an access database that can be read by required Air Force software to generate required reports. The version of the manufacturer software currently in use has been upgraded and the newer version stores data in a postgresql database that cannot be used by reporting sotware.

## _Goal:_
Develop a programatic solution for gathering and reporting laboratory environmental data.

## Import statements 
    * datetime module 
        - used to clean up DateStamp and TimeStamp formating
    * pandas
        - used for data processing
    * numpy
        - to be determined

In [1]:
# import numpy and pandas, and DataFrame / Series
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
from datetime import datetime, timedelta

# Set some pandas options
pd.set_option('display.notebook_repr_html', True)
pd.set_option('display.max_columns', 15)
pd.set_option('display.max_rows', 150)
pd.set_option('display.large_repr', 'truncate')

# And some items for matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
# pd.options.display.mpl_style = 'default'
plt.style.use('fivethirtyeight')

## Import Environmental Data
20170128 - At the moment, there is not a way to collect logger data programatically. I'll be working with csv data generated using the "Readings" table stored in the `LogWare.mdb` and `Logware.mbw` exported from currently used version of the LogWare software. 

* Import data from LogWare database 
   


### Create raw_data frame using csv file
    
The data imported from LogWare database needs a little bit of work.  Ultimately, we want to produce details describing our ability to monitor and maintain our laboratory environment within the specifications directed by T.O. 00-20-14 over a given time period.

    * parse 'DateStamp' and 'TimeStamp' fields to create 'DateTimeStamp' field.
    * Remove original 'DateStamp' and 'TimeStamp' columns
    * use custom *dt_stamp* function instead of lambda


#### Date Parser for `pd.read_csv` data import

In [2]:
# Custom function to combine DateStamp and TimeStamp
def dt_stamp(date, time):
    dt = datetime.strptime(date, "%m/%d/%Y %H:%M:%S")
    tm = datetime.strptime(time,"%m/%d/%Y %H:%M:%S").time()
    dtc = datetime.combine(dt, tm)
    return dtc

In [3]:
# parse 'DateStamp' and 'TimeStamp' fields to create 'DateTimeStamp' field.
# remove original 'DateStamp' and 'TimeStamp' columns
raw_data = pd.read_csv('data/readings.csv', parse_dates={'dts' : ['DateStamp', 'TimeStamp']}, keep_date_col=False, date_parser=dt_stamp)

#### Alt Date Parser for pd.read_csv data import
  * Anonymous function to combine DateStamp and TimeStamp

```python
timestamper = lambda dt, tm: datetime.combine(datetime.strptime(dt,"%m/%d/%Y %H:%M:%S"),datetime.strptime(tm,"%m/%d/%Y %H:%M:%S").time())
```

```python
lab_data_orig = pd.read_csv('data/readings.csv', parse_dates={'DateTimeStamp' : ['DateStamp','TimeStamp']},keep_date_col=True, date_parser=timestamper)

lab_data = lab_data_orig
```

### Cleansing and Data Processing

In [4]:
raw_data.head()

,dts,ID,LocationID,LoggerID,SensorID,UserID,TempReading,HumidReading,TempAlarmMin,TempAlarmMax,HumidAlarmMin,HumidAlarmMax,CRC
0,2014-06-01,5277124,2,2,2,18,19.97,36.2,19.5,20.5,20.0,50.0,2249
1,2014-06-01,5277125,4,3,8,18,21.47,34.6,19.5,26.1,20.0,50.0,2250
2,2014-06-01,5277126,3,4,5,18,22.39,36.8,19.5,26.1,20.0,50.0,2253
3,2014-06-01,5277127,6,5,7,18,20.87,34.3,19.5,26.1,20.0,50.0,2253
4,2014-06-01,5277128,7,6,6,18,21.08,35.0,19.5,26.1,20.0,50.0,2152


#### Extract out only the data we need.

In [5]:
data = raw_data.loc[:, ['dts', 'LocationID', 'TempReading', 'HumidReading', 'TempAlarmMin', 'TempAlarmMax', 'HumidAlarmMin', 'HumidAlarmMax']]

#### Extract out only the requested date range 

In [6]:
def date_range():
    start_date = input('Enter report start date: ')
    start_date = datetime.strptime(start_date, '%m/%d/%Y')
    end_date = input('Enter report end date: ')
    end_date = datetime.strptime(end_date, '%m/%d/%Y')
    return start_date, end_date




start_date, end_date = date_range()
start_date, end_date

Enter report start date: 1/23/2016
Enter report end date: 1/22/2017


(datetime.datetime(2016, 1, 23, 0, 0), datetime.datetime(2017, 1, 22, 0, 0))

In [7]:
# data = data[(data['dts'] >= datetime(2016,1,23)) & (data['dts'] <= datetime(2017,1,23))]
data = data[(data['dts'] >= start_date) & (data['dts'] <= end_date)]

#### Update Temperature and Humidity Requirements based on location

Primary report criteria is based on laboratory environmental requirements. The environmental requirements are determined by location as follows:

* **68° Room**
  * Temperature: 68 ± 1 °F  
  * Relative Humidity: 35% ± 15%
* **Main Laboratory**
  * Temperature: 73 ± 6 °F
  * Relative Humidity: 35% ± 15%

Our dataframe after import from csv contains missing data for these requirements in °C in the following columns:

* *TempAlarmMin*
* *TempAlarmMax*
* *HumidAlarmMin*
* *HumidAlarmMax*

Now is a good time to update these column values in each row to the correct parameters in °F based on the value for *LocationID* of each record in our dataframe. 

* *LocationID* = 2 contain records for the 68° Room 
* All other *LocationID* records represent locations in the main laboratory

In [8]:
data.loc[data.LocationID == 2, ['TempAlarmMin', 'TempAlarmMax', 'HumidAlarmMin', 'HumidAlarmMax']] = 67.0, 69.0, 20.0, 50.0
data.loc[data.LocationID != 2, ['TempAlarmMin', 'TempAlarmMax', 'HumidAlarmMin', 'HumidAlarmMax']] = 67.0, 79.0, 20.0, 50.0

#### Convert 'TempReading' values to °F
We should also convert our recorded temperature readings from °C to °F.  

In [9]:
# Custom function to convert our TempReading values from °C to °F.
def convert_temp(raw_temp):
    return raw_temp * 1.8 + 32

In [10]:
data['TempReading'] = data['TempReading'].apply(convert_temp)

At this point, the dataset is relatively clean, and ready for analysis. We can now begin grouping and aggregating our data to produce the information needed for our report.

In [12]:
data1 = data.copy()
data1.head()

,dts,LocationID,TempReading,HumidReading,TempAlarmMin,TempAlarmMax,HumidAlarmMin,HumidAlarmMax
1038415,2016-01-23,2,67.694,27.7,67.0,69.0,20.0,50.0
1038416,2016-01-23,4,70.700,30.6,67.0,79.0,20.0,50.0
1038417,2016-01-23,3,72.554,28.2,67.0,79.0,20.0,50.0
1038418,2016-01-23,6,69.188,32.3,67.0,79.0,20.0,50.0
1038419,2016-01-23,7,68.126,32.1,67.0,79.0,20.0,50.0


## Check Point -  Clean Data

### Get Time Difference between readings for each location

We need to add timedelta column to the dataframe that provides the elapsed time between each records timestamp and the proceding timestamp based on location. 

The records in our dataset our currently sorted by time. To produce time deltas for each location, we must get the time difference based on location sorted 'dts' column.

**_Our Timestamp column('dts') has already been parsed as datetime object, we'll leverage that to create 'TDelta' column for our data frame._**
```
    In [ ]: data1['dts'].dtype
    Out [ ]:dtype('<M8[ns]')
```


1. *Let's select out our timestamp series **_'dts'_** grouped by **_'LocationID'_**.*
  - **_`gts`_** will be assigned to our selected *grouped time series*

2. *Apply diff method to this grouped time series and pass it back as a new column **_'TDelta'_** to our **_data1_** data frame.* 
  - **_`data1['TDelta']_** = gts.diff().fillna(0) / np.timedelta64(1,'m')` will give the timedelta between each record for each group, fill all *NaT(Null)* values in the series with 0, and finally reduce our timedelta object to and integer that represents each value in minutes.
  

In [20]:
# Group by 'LocationID', and select the 'dts' column:
gts = data1.groupby('LocationID')['dts']
# Create new column in our 
data1['TDelta'] = gts.diff().fillna(0) / np.timedelta64(1,'m')

*Let's take a look at the first 5 records of data1 with the newly added **_'TDelta'_** column.*

In [21]:
data1.head()

,dts,LocationID,TempReading,HumidReading,TempAlarmMin,TempAlarmMax,HumidAlarmMin,HumidAlarmMax,TDelta
1038415,2016-01-23,2,67.694,27.7,67.0,69.0,20.0,50.0,0.0
1038416,2016-01-23,4,70.700,30.6,67.0,79.0,20.0,50.0,0.0
1038417,2016-01-23,3,72.554,28.2,67.0,79.0,20.0,50.0,0.0
1038418,2016-01-23,6,69.188,32.3,67.0,79.0,20.0,50.0,0.0
1038419,2016-01-23,7,68.126,32.1,67.0,79.0,20.0,50.0,0.0


*Great! Our working dataframe **`data1`** now includes the column **`TDelta`** and stores the timedelta values as a float in minutes from previous records **`dts`** value based on each records **`LocationID`**. Our data frame should now contain all of the data necessary to build our report.*

## Part II: All Records Detail Report

### Example: Final Report Dataframe

| LOCATION 	| SPECIFICATION               	| START_DATE 	| END_DATE  	| FIRST_POINT_RECORDED  	| LAST_POINT_RECORDED  	| TOTAL_HOURS_EVALUATED 	| TOTAL_HOURS_RECORDED 	| TOTAL_HOURS_OUT 	| PERCENT_OUT 	| HOURS_TEMP_HIGH 	| HOURS_TEMP_LOW 	| HOURS_RH_HIGH 	| HOURS_RH_LOW 	| HOURS_OVERLAP 	| HOURS_NO_DATA 	| INT_GREATER_THAN_15 	| HRS_DOWN_FOR_MAINT 	| DUPLICATE_RECORDS 	|
|----------	|-----------------------------	|------------	|-----------	|-----------------------	|----------------------	|-----------------------	|----------------------	|-----------------	|-------------	|-----------------	|----------------	|---------------	|--------------	|---------------	|---------------	|---------------------	|--------------------	|-------------------	|
| QA       	| Temp 73 ± 6° F RH 35% ± 15% 	| 1/23/2016  	| 1/22/2017 	| 1/23/2016 12:00:00 AM 	| 1/17/2017 8:45:00 AM 	| 8784.00               	| 8647.67              	| 429.17          	| 4.89 %      	| 1.75            	| 3.67           	| 2.58          	| 284.92       	| 0.08          	| 136.33        	| 1                   	| 0.00               	| 0                 	|
| K3       	| Temp 73 ± 6° F RH 35% ± 15% 	| 1/23/16    	| 1/22/17   	| 1/23/16 0:00          	| 1/17/17 8:40         	| 8784                  	| 8647.58              	| 183.17          	| 2.09%       	| 3.5             	| 1.5            	| 0.33          	| 41.42        	| 0             	| 136.42        	| 1                   	| 0                  	| 0                 	|
| 68       	| Temp 68 ± 1° F RH 35% ± 15% 	| 1/23/16    	| 1/22/17   	| 1/23/16 0:00          	| 1/17/17 8:40         	| 8784                  	| 8647.58              	| 237.41          	| 2.70%       	| 10.33           	| 1.83           	| 45.42         	| 43.58        	| 0.17          	| 136.42        	| 1                   	| 0                  	| 0                 	|
|          	|                             	|            	|           	|                       	|                      	|                       	|                      	|                 	|             	|                 	|                	|               	|              	|               	|               	|                     	|                    	|                   	|
|          	|                             	|            	|           	|                       	|                      	|                       	|                      	|                 	|             	|                 	|                	|               	|              	|               	|               	|                     	|                    	|                   	|
|          	|                             	|            	|           	|                       	|                      	|                       	|                      	|                 	|             	|                 	|                	|               	|              	|               	|               	|                     	|                    	|                   	|

### Group our entire dataframe by LocationID

In [22]:
location = {2 : 'K6C', 3 : 'K8', 4 : 'K3', 5 : 'QA', 6 : 'K1', 7 : 'K6', 8 : 'Spare'}

In [23]:
data1['LocationID'] = data1['LocationID'].map(location)

In [24]:
data1.head(10)

,dts,LocationID,TempReading,HumidReading,TempAlarmMin,TempAlarmMax,HumidAlarmMin,HumidAlarmMax,TDelta
1038415,2016-01-23,K6C,67.694,27.7,67.0,69.0,20.0,50.0,0.0
1038416,2016-01-23,K3,70.700,30.6,67.0,79.0,20.0,50.0,0.0
1038417,2016-01-23,K8,72.554,28.2,67.0,79.0,20.0,50.0,0.0
1038418,2016-01-23,K1,69.188,32.3,67.0,79.0,20.0,50.0,0.0
1038419,2016-01-23,K6,68.126,32.1,67.0,79.0,20.0,50.0,0.0


In [69]:
data1.groupby('LocationID').size()


LocationID
K1     103774
K3     103773
K6     103773
K6C    103772
K8     103772
QA     103774
dtype: int64

In [47]:
def report_stats = {
    'dts': {
        'first record': 'min',
        'last record' : 'max'
    }
}

In [67]:
temp_bins = [0,67.0,79.0, 1000]
temp_groups= ['low', 'good', 'high']



dts              622638
LocationID       622638
TempReading      622638
HumidReading     622638
TempAlarmMin     622638
TempAlarmMax     622638
HumidAlarmMin    622638
HumidAlarmMax    622638
TDelta           622638
dtype: int64

In [74]:
print(data1.groupby('LocationID').count())

               dts  TempReading  HumidReading  TempAlarmMin  TempAlarmMax  \
LocationID                                                                  
K1          103774       103774        103774        103774        103774   
K3          103773       103773        103773        103773        103773   
K6          103773       103773        103773        103773        103773   
K6C         103772       103772        103772        103772        103772   
K8          103772       103772        103772        103772        103772   
QA          103774       103774        103774        103774        103774   

            HumidAlarmMin  HumidAlarmMax  TDelta  
LocationID                                        
K1                 103774         103774  103774  
K3                 103773         103773  103773  
K6                 103773         103773  103773  
K6C                103772         103772  103772  
K8                 103772         103772  103772  
QA                 103774 

In [38]:
data1.iloc[:1]['dts'].dt.weekday

1038415    5
Name: dts, dtype: int64

In [ ]:
# How many rows the dataset
data1['LocationID'].count()

In [ ]:
# What was the longest TDelta data entry?
data1['TDelta'].max()

In [ ]:
# How many minutes of TempAlarmMin == 67.0 are recorded in total?
data1['TDelta'][data1['TempAlarmMin'] == 67.0].sum()

In [ ]:
# How many entries are there for each location?
data1['LocationID'].value_counts()

In [ ]:
# Number of non-null unique TempReading entries
data1['TempReading'].nunique()

#### Summarising Groups in the DataFrame

In [ ]:
data1.groupby('LocationID').groups.keys()

In [ ]:
len(data1.groupby('LocationID').groups['K1'])

In [ ]:
# Get the first entry for each location
data1.groupby('LocationID').first()

In [ ]:
# Get the sum of the durations per location
data1.groupby('LocationID')['TDelta'].sum() / 60

In [ ]:
# Get the number of datetimestamps / entries in each location
data1.groupby('LocationID')['dts'].count()

In [ ]:
# What is the sum of durations, for TempReadings greater than TempAlarmMax for each location
data1[data1['TempReading'] > data1['TempAlarmMax']].groupby('LocationID')['TDelta'].sum()

In [ ]:
# Group the data frame by location and item and extract a number of stats from each group
data1.groupby('LocationID').agg({'Specification': 'String',      # Describe the Temp and Humidity specs for the location
                                 'Start Date': 'start_date',     # Insert the report start date variable from user
                                 'End Date': 'end_date',         # insert the report start date variable from user
                                })    # get the first date per group

In [ ]:
data1['TDelta'][data1['TempReading'] > data1['TempAlarmMax']].sum()

In [ ]:
for k, group in grouped:
    eval = []
    td = group.index.max() - group.index.min()
    td = td.total_seconds()/3600
    print(k,td)

In [ ]:
# group by LocationID, and select the dts column:
gs = data1.groupby('LocationID')['dts']
for k in gs:
    print(k)
    print(g)
    

### Total Hours Evaluated
Report provides information for the number of total hours evaluated of over a user determined time span.
    
    * send user provided start and end dates to custom function that returns total hours evaluated

In [ ]:
data1

In [ ]:
data1['TimeDiff'] = gs.diff().fillna(0) / np.timedelta64(1,'m')
data1.TimeDiff.dtype

In [ ]:
data1.loc[data1['TimeDiff'] !=5.0]

In [ ]:
# end_date = datetime(2017, 1, 22, 0, 0, 0)

# Custom function to provide timedelta for entire requested date range
def hours_eval(start, end):
    return (end_date - start_date).total_seconds() / 3600

hours_eval(start_date, end_date)

In [ ]:
grouped['deltaT'] = grouped.index.to_series().diff().dt.seconds.div(60, fill_value=0)

### Step 2: Filter date range dataframe by selected location and create a new dataframe

#### Step 2A: Select rows from a DataFrame based on value in 'LocationID'



In [ ]:
qa_data = data.loc[labdates['LocationID'] == 5]
qa_data.head()

### Step 3: Gather report data

#### Step 3A: Get first and last point recorded

In [ ]:
qa_data.index = qa_data['DateTimeStamp']
del qa_data['DateTimeStamp']
qa_data

In [ ]:
qa_min_date = qa_data.index.min()
qa_max_date = qa_data.index.max()
print(qa_min_date, qa_max_date)

In [ ]:
printqa_min_date


In [ ]:
lab_data['DateTimeStamp'].dtype

In [ ]:
lab_data.describe()

###  Validate imported data

###### Inspect data

In [ ]:
lab_data

###### Inspect columns

In [ ]:
lab_data.columns

###### Inspect shape

In [ ]:
lab_data.shape

###### Create dataframe for missing values

In [ ]:
null_data = lab_dtc[lab_dtc.isnull().any(axis=1)]
null_data

###### Update Temperature and Humidity criteria based on location

Primary report criteria is based on laboratory environmental requirements. The environmental requirements are determined by location as follows:

* **68° Room**
  * Temperature: 68 ± 1 °F  
  * Relative Humidity: 35% ± 15%
* **Main Laboratory**
  * Temperature: 73 ± 6 °F
  * Relative Humidity: 35% ± 15%

Our dataframe after import from csv contains incomplete data for these requirements in °C in the following columns:

* *TempAlarmMin*
* *TempAlarmMax*
* *HumidAlarmMin*
* *HumidAlarmMax*

Now is a good time to update the values in each of these columns to the correct parameters based on *LocationID* of each record in our dataframe. 

* *LocationID* = 2 contain records for the 68° Room 
* All other *LocationID* records represent locations in the main laboratory



In [ ]:
lab_data['LocationID'].unique()

In [ ]:
lab_data.loc[lab_data.LocationID == 2, ['TempAlarmMin', 'TempAlarmMax', 'HumidAlarmMin', 'HumidAlarmMax']] = 67.0, 69.0, 20.0, 50.0
lab_data.loc[lab_data.LocationID != 2, ['TempAlarmMin', 'TempAlarmMax', 'HumidAlarmMin', 'HumidAlarmMax']] = 67.0, 79.0, 20.0, 50.0

### Convert Temperatures to °F 

###### Function to convert °C to °F

In [ ]:
def convert_temp(raw_temp):
    return raw_temp * 1.8 + 32

In [ ]:
lab_data['TempReading'] = lab_data['TempReading'].apply(convert_temp)

lab_data

In [ ]:
lab_data_clean = lab_data

In [ ]:
lab_data.describe()

In [ ]:
lab_data.DateTimeStamp.dtype

In [ ]:
mask = (lab_data['DateTimeStamp'] > '2016-1-27') & (lab_data['DateTimeStamp'] <= '2017-1-27')

In [ ]:
lab_data.loc[mask].plot()

In [ ]:
lab_data.columns

In [ ]:
lab_data[['TempReading', 'HumidReading', 'TempAlarmMin',
       'TempAlarmMax', 'HumidAlarmMin', 'HumidAlarmMax']].plot()

In [ ]:
lab_data2 = lab_data

In [ ]:
bylocation = lab_data.groupby('LocationID')

In [ ]:
bylocation[['TempReading', 'HumidReading']].agg([np.min, np.max, np.mean])

In [ ]:
locations.describe()

In [ ]:
def subset_loc(df_raw, locations):
    loc_frame = None
    for loc in locations:
        loc_frame = df_raw[df_raw.LocationID.isin(locations)]
    print(loc_frame)
    return loc_frame

subset_loc(lab_data, locs)
        

In [ ]:
k6c_data = lab_data[lab_data.LocationID == 2]

In [ ]:
k6c_data.dtypes

In [ ]:
first_date = k6c_data.DateTimeStamp.values[6]
first_date

In [ ]:
locations = pd.read_csv('data/Locations.csv')
location = locations.set_index('LocationID')['Name'].to_dict()
location

In [ ]:
def get_loc(key, default=None):
    k = int(key)
    if k in location:
        return location[k]
    else:
        return default

In [ ]:
loggers = pd.read_csv('data/Loggers.csv')

In [ ]:
loggers = pd.read_csv('data/Loggers.csv')
logger = loggers.set_index('LoggerID')['Serial'].to_dict()
logger

In [ ]:
def get_logger(key, default=None):
    k = int(key)
    if k in location:
        return logger[k]
    else:
        return default

In [ ]:
sensors = pd.read_csv('data/Sensors.csv')
sensor = sensors.set_index('SensorID')['Serial'].to_dict()
sensor

In [ ]:
def get_sensor(key, default=None):
    k = int(key)
    if k in location:
        return sensor[k]
    else:
        return default

In [ ]:
k6c_data

In [ ]:
loggers

In [ ]:
lab_data

In [ ]:
x = lab_data.ix[500, 'DateTimeStamp']

In [ ]:
x = pd.to_datetime(x)
x

In [ ]:
lab_data2 = lab_data

In [ ]:
mask = (lab_data2['DateTimeStamp'] > '2017-1-1') & (lab_data2['DateTimeStamp'] <= '2017-1-14')

In [ ]:
working = lab_data2.loc[mask]

In [ ]:
wd = working.groupby('LocationID')

In [ ]:
wd

In [ ]:
wd[['TempReading', 'HumidReading']].agg([np.min, np.max, len])

In [ ]:
wd.describe()

In [ ]:
d={'item1':{'major0':{'minor0':1,
                      'minor1':2},
            'major1':{'minor0':3,
                      'minor1':4}
           },
   'item2':{'major0':{'minor0':5,
                      'minor1':6},
            'major1':{'minor0':7,
                      'minor1':8}
           }
  }
d

In [ ]:
p = pd.Panel(d)

In [ ]:
p